In [9]:
import requests

def send_motor_command(option):
    # url = "http://192.168.100.150/sendOption"  # Địa chỉ IP của ESP8266
    url = "http://192.168.89.168/sendOption"  # Địa chỉ IP của ESP8266
    data = {'option': option}  # Dữ liệu gửi đi, là tùy chọn bật/tắt động cơ

    try:
        # Gửi yêu cầu POST đến ESP8266
        response = requests.post(url, data=data)
        # Kiểm tra phản hồi từ server
        if response.status_code == 200:
            print(option)
        else:
            print("Có lỗi xảy ra khi gửi yêu cầu:", response.status_code)
    except requests.exceptions.RequestException as e:
        print("Không thể kết nối đến ESP8266:", e)

# Ví dụ gọi hàm để bật động cơ (option = 1)

# Ví dụ gọi hàm để tắt động cơ (option = 2)
# send_motor_command(2)

In [10]:
send_motor_command(1) # quạt 

1


In [11]:
send_motor_command(2)

2


In [5]:
send_motor_command(3) # cửa

Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 3


In [6]:
send_motor_command(4)

Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 4


In [20]:
send_motor_command(5) # phòng ngủ (bedroom)


5


In [13]:
send_motor_command(6)

6


In [19]:
send_motor_command(7) # (Dining room)

7


In [15]:
send_motor_command(8)

8


In [18]:
send_motor_command(9) # (Living room)

9


In [17]:
send_motor_command(10) # 

10


In [15]:
import time

for j in range(1, 11):  # Vòng lặp từ 1 đến 10
    for i in range(1, 11):  # Vòng lặp từ 1 đến 10
        send_motor_command(i)
        time.sleep(1)  # Delay 1 giây


Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 1
Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 2
Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 3
Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 4
Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 5
Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 6
Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 7
Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 8
Có lỗi xảy ra khi gửi yêu cầu: 400
Có lỗi xảy ra khi gửi yêu cầu: 400
Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 1
Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 2
Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 3
Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 4
Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 5
Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 6
Gửi thành công, Động cơ đã được điều khiển theo tùy chọn: 7
Gửi thành công, Động cơ đã đượ

# v1

In [ ]:
// ĐỘNG CƠ DC
#include <ESP8266WiFi.h>
#include <ESP8266WebServer.h>

// Khai báo các chân điều khiển động cơ và LED
const int motorPin1 = D6; // Chân IN1 của mạch điều khiển động cơ
const int motorPin2 = D5; // Chân IN2 của mạch điều khiển động cơ
const int ledPin1 = D1;   // Chân kết nối LED 1
const int ledPin2 = D2;   // Chân kết nối LED 2
const int ledPin3 = D3;   // Chân kết nối LED 3

ESP8266WebServer server(80); // Khởi tạo server trên cổng 80

void setup() {
  Serial.begin(9600);
  delay(1000);

  // Kết nối WiFi
  WiFi.begin("OPPO Reno VanManh Pro Vip", "vanmanh999"); // SSID và mật khẩu
  while (WiFi.waitForConnectResult() != WL_CONNECTED) {
    Serial.println("Connecting...");
  }
  Serial.println("IP: ");
  Serial.println(WiFi.localIP());

  // Thiết lập các chân điều khiển động cơ và LED là output
  pinMode(motorPin1, OUTPUT);
  pinMode(motorPin2, OUTPUT);
  pinMode(ledPin1, OUTPUT);
  pinMode(ledPin2, OUTPUT);
  pinMode(ledPin3, OUTPUT);

  // Thiết lập server
  server.on("/", [] {
    server.send(200, "text/html",
                "<!DOCTYPE html>"
                "<html>"
                "<head>"
                "<title>Control DC Motor and LED</title>"
                "<script>"
                "function sendOption() {"
                "var optionInput = document.getElementById('option');"
                "var xhttp = new XMLHttpRequest();"
                "var option = optionInput.value;"
                "xhttp.open('POST', 'sendOption', true);"
                "xhttp.setRequestHeader('Content-type', 'application/x-www-form-urlencoded');"
                "xhttp.send('option=' + option);"
                "}"
                "</script>"
                "</head>"
                "<body>"
                "<label>Option (1: Engine On, 2: Engine Off, 3: LED1 On, 4: LED1 Off, 5: LED2 On, 6: LED2 Off, 7: LED3 On, 8: LED3 Off): </label>"
                "<input type='text' id='option'>"
                "<input type='button' value='Send' onclick='sendOption()'>"
                "</body>"
                "</html>");
  });

  setupRoutes(); // Thiết lập các route của server
  server.begin(); // Khởi động server
}

void handleSendOption() {
  if (server.hasArg("option")) {
    int option = server.arg("option").toInt();
    switch (option) {
      case 1:
        // Bật động cơ
        digitalWrite(motorPin1, HIGH);
        digitalWrite(motorPin2, LOW);
        Serial.println("Motor ON");
        server.send(200, "text/plain", "Motor turned ON");
        break;
      case 2:
        // Tắt động cơ
        digitalWrite(motorPin1, LOW);
        digitalWrite(motorPin2, LOW);
        Serial.println("Motor OFF");
        server.send(200, "text/plain", "Motor turned OFF");
        break;
      case 3:
        // Bật LED 1
        digitalWrite(ledPin1, HIGH);
        Serial.println("LED 1 ON");
        server.send(200, "text/plain", "LED 1 turned ON");
        break;
      case 4:
        // Tắt LED 1
        digitalWrite(ledPin1, LOW);
        Serial.println("LED 1 OFF");
        server.send(200, "text/plain", "LED 1 turned OFF");
        break;
      case 5:
        // Bật LED 2
        digitalWrite(ledPin2, HIGH);
        Serial.println("LED 2 ON");
        server.send(200, "text/plain", "LED 2 turned ON");
        break;
      case 6:
        // Tắt LED 2
        digitalWrite(ledPin2, LOW);
        Serial.println("LED 2 OFF");
        server.send(200, "text/plain", "LED 2 turned OFF");
        break;
      case 7:
        // Bật LED 3
        digitalWrite(ledPin3, HIGH);
        Serial.println("LED 3 ON");
        server.send(200, "text/plain", "LED 3 turned ON");
        break;
      case 8:
        // Tắt LED 3
        digitalWrite(ledPin3, LOW);
        Serial.println("LED 3 OFF");
        server.send(200, "text/plain", "LED 3 turned OFF");
        break;
      default:
        server.send(400, "text/plain", "Invalid option");
    }
  } else {
    server.send(400, "text/plain", "Missing 'option' parameter");
  }
}

void setupRoutes() {
  server.on("/sendOption", handleSendOption); // Route để xử lý tùy chọn điều khiển
}

void loop() {
  server.handleClient(); // Xử lý các yêu cầu từ client
}
